In [ ]:
import warnings
import pandas as pd
import json
import re
from collections import Counter
# from sklearn.model_selection import train_test_split
import logging
datPath = '/root/autodl-tmp/BertClassNews/data/'

warnings.filterwarnings("ignore")

# Read data

In [2]:
import pandas as pd
datPath = '/root/autodl-tmp/BertClassNews/data/'
wtPath = '/root/autodl-tmp/BertClassNews/wt/'

train_df = pd.read_json(datPath + 'train_cls.json')
val_df = pd.read_json(datPath + 'val_cls.json')
test_df = pd.read_json(datPath + 'test_cls.json')

# train_df = pd.read_json(datPath + 'smpl_train.json')
# val_df = pd.read_json(datPath + 'smpl_val.json')
# test_df = pd.read_json(datPath + 'smpl_test.json')

lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}
train_df['tag'] = train_df['tag'].map(lblEncode)
val_df['tag'] = val_df['tag'].map(lblEncode)
test_df['tag'] = test_df['tag'].map(lblEncode)

In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# Load model

In [4]:
# !pip install torch transformers
from transformers import BertTokenizer, BertForSequenceClassification

# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# lblEncode = {j:i for i,j in zip(range(len(train_df['tag'].unique())), train_df['tag'].unique())}

model = BertForSequenceClassification.from_pretrained('bert-base-chinese',num_labels=len(lblEncode))
# inputs = tokenizer(text, return_tensors='pt', padding=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}

In [6]:
lblEncode

{'本地旅游': 0,
 '通报查处': 1,
 '基建民生': 2,
 '社会热点': 3,
 '暖新闻': 4,
 '人事任免': 5,
 '政策类型': 6,
 '产业金融': 7,
 '人文历史': 8,
 '数据排名': 9}

In [ ]:
clla

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import sklearn.model_selection
from torch.nn import CrossEntropyLoss

In [8]:
# reverse_lblEncode = {lblEncode[i]:i for i in lblEncode}

class BERTDataset(Dataset):

    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_texts, val_texts, test_texts = train_df['content'].tolist(), val_df['content'].tolist(), test_df['content'].tolist()
train_labels, val_labels, test_labels = train_df['tag'].tolist(), val_df['tag'].tolist(), test_df['tag'].tolist()

# train_texts, val_texts, test_texts = train_df['content'].iloc[:100].tolist(), val_df['content'].iloc[:100].tolist(), test_df['content'].iloc[:100].tolist()
# train_labels, val_labels, test_labels = train_df['tag'].iloc[:100].tolist(), val_df['tag'].iloc[:100].tolist(), test_df['tag'].iloc[:100].tolist()
def generateDataloader(df, lbl, BATCH_SIZE = 16):
    inputs = tokenizer(df, padding=True, truncation=True,return_tensors='pt')
    dataset = BERTDataset(inputs, lbl)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    return dataloader
import time
import numpy as np

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def evaluate(model, dataloader):
    model.eval()
    final_pred = []
    total = 0
    correct = 0
    pred = {i:[] for i in range(len(lblEncode))}
    # pred = {i:[] for i in [0,2,3]}
    with torch.no_grad():

        for batch in dataloader:
            inputs = torch.stack([t for t in batch['input_ids']]).to(device)
            labels = torch.tensor(batch['labels']).to(device)
            attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
            outputs = model(input_ids=inputs, attention_mask=attention_mask,
                      labels=labels)
            predictions = outputs[1].argmax(dim=1)
            for i in range(len(predictions)):
                predRes = predictions[i].item()
                pred[predRes].append((predictions[i] == labels[i]).item())
                final_pred.append((reverse_lblEncode[predRes], reverse_lblEncode[labels[i].item()]))
                # final_pred.append((predRes, labels[i].item()))

            correct += (predictions == labels).sum().item()
            total += labels.shape[0]
      
    accuracy = correct/total
    res = {reverse_lblEncode[i]:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }
  # res = {i:sum(pred[i])/len(pred[i]) if len(pred[i]) > 0 else np.nan for i in pred }

    return accuracy, res, final_pred

def model_train(model, train_dataloader, val_dataloader, iter = 3):
    for epoch in range(3):
      start = time.time()
      losses = 0
      model.train()
      idx = 0
      for batch in train_dataloader:
        inputs, token_type_ids, attention_mask, labels = batch['input_ids'], batch['token_type_ids'], batch['attention_mask'], batch['labels']

        input_ids = torch.stack([t for t in batch['input_ids']]).to(device)

        attention_mask =torch.stack([t for t in batch['attention_mask']]).to(device)
        labels = torch.tensor(batch['labels']).to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,labels=labels)
        loss = criterion(outputs['logits'],labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses += loss.item()
        acc = (outputs['logits'].argmax(1) == labels).float().mean()
        idx += 1
        curr = time.time()
        timeConsume = curr - start
        if idx % 300 == 0:
            print (f'Epoch {epoch+1}, batch {idx}, so far takes {timeConsume}')
          # los  =outputs['loss']
            print (f'Loss: {losses}')

      print (acc)
      val_acc, val_cat_acc, val_final_pred= evaluate(model, val_dataloader)
      print ('val acc:', val_acc)
        # if idx % 10 == 0:
        #     logging.info(f"Epoch: {epoch}, Batch[{idx}/{len(train_iter)}], "
        #                   f"Train loss :{loss.item():.3f}, Train acc: {acc:.3f}")

      print(f'Epoch {epoch+1} complete, so far takes {timeConsume}')
    return model


def save_model(model, datPath, model_name = 'BertOrigmodelAllDat_v0.pt'):
    # datPath = '/root/wt/'
    # model_save_path  = datPath + 'BertOrigmodelAllDat_v0.pt'
    model_save_path  = datPath + model_name
    if os.path.exists(model_save_path):
        loaded_paras = torch.load(model_save_path)
        model.load_state_dict(loaded_paras)
        logging.info("## 成功载入已有模型，进行追加训练......")
    else:
        torch.save(model.state_dict(), model_save_path)
    return model

## Prepare for new dataset

In [9]:
train_dataloader = generateDataloader(train_df['content'].tolist(), train_df['tag'].tolist()) 
val_dataloader = generateDataloader(val_df['content'].tolist(), val_df['tag'].tolist()) 
test_dataloader = generateDataloader(test_df['content'].tolist(), test_df['tag'].tolist()) 

In [11]:
from imblearn.over_sampling import RandomOverSampler

def remove_part_fPred_train(train_df, train_pred):
    cond_acc = []
    for i,j in train_pred:
        cond_acc.append((i==j) |((i!=j)&(j not in ['本地旅游','基建民生','社会热点'])))
    # cond_acc = pd.Series(cond_acc)
    new_train = train_df[cond_acc]
    new_train = new_train.reset_index(drop = True)
    return new_train

def augment_minority_randomSample(train_df, augment_rate):
    oversample = RandomOverSampler(sampling_strategy = augment_rate)
    X_over, y_over = oversample.fit_resample(train_df.drop('tag', axis=1), train_df['tag'])
    X_over['tag'] = y_over
    return X_over

In [11]:
new_model = save_model(model, wtPath, model_name = 'BertRetrainNewAllDat_v1.pt')

In [12]:
test_accuracy, test_cat_acc, test_pred = evaluate(new_model, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)

Test Accuracy: 0.8826125330979699 Test Cat Accuracy: {'本地旅游': 0.9641255605381166, '通报查处': 0.8914893617021277, '基建民生': 0.8492723492723493, '社会热点': 0.9102112676056338, '暖新闻': 0.6333333333333333, '人事任免': 0.9675090252707581, '政策类型': 0.8978328173374613, '产业金融': 0.7604790419161677, '人文历史': 0.423728813559322, '数据排名': 0.6666666666666666}


# Train based on augmented 

In [13]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from nlpcda import Randomword
from nlpcda import Similarword
from nlpcda import Homophone
from nlpcda import RandomDeleteChar
from nlpcda import Ner
from nlpcda import CharPositionExchange
from nlpcda import baidu_translate
from nlpcda import EquivalentChar


def test_Randomword(test_str, create_num=3, change_rate=0.3):
    '''
    随机【（等价）实体】替换，这里是extdata/company.txt ，随机公司实体替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    smw = Randomword(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_Similarword(test_str, create_num=3, change_rate=0.3):
    '''
    随机【同义词】替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    smw = Similarword(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_Homophone(test_str, create_num=3, change_rate=0.1):
    '''
    随机【同意/同音字】替换
    :param test_str: 替换文本
    :param create_num: 增强为多少个
    :param change_rate: 文本变化率/最大多少比例会被改变
    :return:
    '''
    hoe = Homophone(create_num=create_num, change_rate=change_rate)
    return hoe.replace(test_str)


def test_RandomDeleteChar(test_str, create_num=3, change_rate=0.1):
    smw = RandomDeleteChar(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)



def test_ner():
    ner = Ner(ner_dir_name='../write',
              ignore_tag_list=['O', 'T'],
              data_augument_tag_list=['Cause', 'Effect'],
              augument_size=3, seed=0)
    data_sentence_arrs, data_label_arrs = ner.augment('../write/1.txt')
    print(data_sentence_arrs, data_label_arrs)


def test_CharPositionExchange(test_str, create_num=10, change_rate=0.5):
    smw = CharPositionExchange(create_num=create_num, change_rate=change_rate)
    return smw.replace(test_str)


def test_baidu_translate():
    a = 'Free translation for each platform'
    s = baidu_translate(a, appid='xxx', secretKey='xxx')
    print(s)


def test_EquivalentChar(test_str, create_num=10, change_rate=0.5):
    s = EquivalentChar(create_num=create_num, change_rate=change_rate)
    return s.replace(test_str)


def test():
    ts = '''这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击'''
    rs1 = test_Randomword(ts)
    rs2 = test_Similarword(ts)
    rs3 = test_Homophone(ts)
    rs4 = test_RandomDeleteChar(ts)
    rs5 = test_EquivalentChar(ts)
    print('随机实体替换>>>>>>')
    for s in rs1:
        print(s)
    print('随机近义词替换>>>>>>')
    for s in rs2:
        print(s)
    print('随机近义字替换>>>>>>')
    for s in rs3:
        print(s)

    print('随机字删除>>>>>>')
    for s in rs4:
        print(s)
    print('等价字替换>>>>>>')
    for s in rs5:
        print(s)


# if __name__ == '__main__':
#     # ts = '''今天是2020年3月8日11:40，天气晴朗，天气很不错。'''
#     # rs = EquivalentChar(create_num=3, change_rate=0.5)
#     # res = rs.replace(ts)
#     # print('等价字替换>>>>>>')
#     # for s in res:
#     #     print(s)
    # test()

Simbert不能正常使用，除非你安装：bert4keras、tensorflow ，为了安装快捷，没有默认安装.... No module named 'bert4keras'


In [10]:
from IPython.utils import io
# {'本地旅游': 0,
#  '通报查处': 1,
#  '基建民生': 2,
#  '社会热点': 3,
#  '暖新闻': 4,
#  '人事任免': 5,
#  '政策类型': 6,
#  '产业金融': 7,
#  '人文历史': 8,
#  '数据排名': 9}
augment_rate2 = {0: 5331,1: 2477,2: 5072,3: 6663,4: 544 * 2,5: 1734,6: 2046,7: 908*2,8: 267*3,9: 639*2}
# new_train = remove_part_fPred_train(train_df, train_pred)
new_train = pd.read_json(datPath + 'new_train_cls.json')
# new_train = augment_minority_randomSample(new_train, augment_rate2)
# new_train_dataloader = generateDataloader(new_train['content'].tolist(), new_train['tag'].tolist()) 

In [58]:

augment_nlpcda(tmp[17][0], equalNum = 1)[0]

load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/company.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/同义词.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/同音意字.txt done
load :/root/miniconda3/lib/python3.8/site-packages/nlpcda/data/等价字.txt done


'雨天黔西南男子骑摩托被困激流中，私家车挡住激流，众人冒雨扶车。。'

In [60]:
# res1 = [i[0] for i in tmp]

# [j[1] for j in tmp]
# len(res1), len(tmp), train_aug.shape

res1, res2 = [], []
with io.capture_output() as captured:
    for i in tmp:
        if len(i) == 2:
            res1.append(i[0])
            res2.append(i[1])
        elif len(i) == 1:
            res1.append(i[0])
            res2.append(augment_nlpcda(i[0], equalNum = 1)[0])

In [69]:
augment_rate2 = {0: 5331,1: 2477,2: 5072,3: 6663,4: 544 * 2,5: 1734,6: 2046,7: 908*2,8: 267*3,9: 639*2}
def augment_nlpcda(ts, equalNum = 2):
    smw1 = Randomword(create_num=1, change_rate=0.3)
    smw2 = Similarword(create_num=1, change_rate=0.3)
    hoe = Homophone(create_num=1, change_rate=0.1)
    smw3 = RandomDeleteChar(create_num=1, change_rate=0.1)
    s = EquivalentChar(create_num=equalNum, change_rate=0.5)
    # return s.replace(ts)[0]
    res = s.replace(ts)
    return [smw1.replace(smw2.replace(hoe.replace(smw3.replace(i)[0])[0])[0])[0] for i in res]

def augment_minority_augmentSample(train_df, augment_rate):
    oversample = RandomOverSampler(sampling_strategy = augment_rate)
    X_over, y_over = oversample.fit_resample(train_df.drop('tag', axis=1), train_df['tag'])
    X_over['tag'] = y_over
    return X_over
train_aug = train_df[train_df['tag'].isin([4,7,8,9])]
with io.capture_output() as captured:
    train_aug1 = train_aug.copy()
    train_aug2 = train_aug.copy()
    tmp = train_aug['content'].apply(lambda x: augment_nlpcda(x)).tolist()
res1, res2 = [], []
with io.capture_output() as captured:
    for i in tmp:
        if len(i) == 2:
            res1.append(i[0])
            res2.append(i[1])
        elif len(i) == 1:
            res1.append(i[0])
            res2.append(augment_nlpcda(i[0], equalNum = 1)[0])
train_aug1['content'] = res1
train_aug2['content'] = res2
train_aug_final = pd.concat([train_df, train_aug1, train_aug2],axis=0).sample(frac=1)

In [68]:
train_aug_final.to_json(datPath+'new_train_nlpcda.jason')

In [70]:
train_aug_final_dataloader = generateDataloader(train_aug_final['content'].tolist(), train_aug_final['tag'].tolist()) 

In [71]:
new_model_v4 = model_train(new_model, train_aug_final_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model_v4, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model_v4 = save_model(new_model_v4, wtPath, model_name = 'BertRetrainNewAllDatRemErrTrainNlpcdaAug_v4.pt')

Epoch 1, batch 300, so far takes 102.95491433143616
Loss: 49.14178957277909
Epoch 1, batch 600, so far takes 205.76029801368713
Loss: 95.65615228097886
Epoch 1, batch 900, so far takes 308.6040370464325
Loss: 144.49581548944116
Epoch 1, batch 1200, so far takes 411.64946126937866
Loss: 195.18263155594468
Epoch 1, batch 1500, so far takes 514.8050725460052
Loss: 241.99835541006178
Epoch 1, batch 1800, so far takes 617.715742111206
Loss: 298.6685672462918
tensor(1., device='cuda:0')
val acc: 0.8852083651350939
Epoch 1 complete, so far takes 663.1937487125397
Epoch 2, batch 300, so far takes 102.77483129501343
Loss: 41.76007594168186
Epoch 2, batch 600, so far takes 205.87410306930542
Loss: 76.46334711485542
Epoch 2, batch 900, so far takes 308.83878898620605
Loss: 114.0493159990292
Epoch 2, batch 1200, so far takes 411.73110246658325
Loss: 150.53454364836216
Epoch 2, batch 1500, so far takes 514.8581447601318
Loss: 189.45315896021202
Epoch 2, batch 1800, so far takes 617.9172775745392
Lo

In [72]:
new_model_v5 = model_train(new_model_v4, train_aug_final_dataloader, val_dataloader)
test_accuracy, test_cat_acc, test_pred = evaluate(new_model_v5, test_dataloader)
print("Test Accuracy:", test_accuracy, "Test Cat Accuracy:", test_cat_acc)
new_model_v5 = save_model(new_model_v5, wtPath, model_name = 'BertRetrainNewAllDatRemErrTrainNlpcdaAug_v5.pt')

Epoch 1, batch 300, so far takes 102.95349550247192
Loss: 35.51844134321436
Epoch 1, batch 600, so far takes 205.94252490997314
Loss: 76.25066650775261
Epoch 1, batch 900, so far takes 308.9322326183319
Loss: 111.10925501096062
Epoch 1, batch 1200, so far takes 411.9672613143921
Loss: 145.50120523478836
Epoch 1, batch 1500, so far takes 514.8288156986237
Loss: 175.0374473250704
Epoch 1, batch 1800, so far takes 617.9021968841553
Loss: 207.4173150168499
tensor(1., device='cuda:0')
val acc: 0.8871927949931309
Epoch 1 complete, so far takes 663.3985121250153
Epoch 2, batch 300, so far takes 103.1886100769043
Loss: 29.015644358936697
tensor(1., device='cuda:0')
val acc: 0.8878033887956037
Epoch 2 complete, so far takes 664.2198729515076
Epoch 3, batch 300, so far takes 103.32352805137634
Loss: 32.25105999573134
Epoch 3, batch 600, so far takes 206.35923600196838
Loss: 61.50817705481313
Epoch 3, batch 900, so far takes 309.558434009552
Loss: 89.56576520064846
Epoch 3, batch 1200, so far tak